In [2]:
import pandas as pd
import os
import re

permutations = [['ORG', 'PER', 'CVL', 'DAT', 'LOC', 'QNT'],
               ['DAT', 'QNT', 'PER', 'LOC', 'ORG', 'CVL'],
               ['CVL', 'LOC', 'ORG', 'QNT', 'DAT', 'PER'],
               ['QNT', 'ORG', 'DAT', 'PER', 'CVL', 'LOC'],
               ['LOC', 'CVL', 'QNT', 'ORG', 'PER', 'DAT'],
               ['PER', 'DAT', 'LOC', 'CVL', 'QNT', 'ORG']]

current_path = './train_data'
for perm in range(1, 7):
    for step in range(1,6):
        for k in range(step+1):
            df = pd.read_csv(current_path+'/perm_'+str(perm)+'/d'+str(k)+'_'+permutations[perm-1][k]+'.csv')
            label = df['label'].copy()
            doc = df['doc'].copy()

            past_ents = permutations[perm-1][:step+1]
            for search_perm in range(1,7):
                if perm == search_perm:
                    continue
                elif permutations[search_perm-1][k] in past_ents:
                    temp = pd.read_csv(current_path+'/perm_'+str(search_perm)+'/d'+str(k)+'_'+permutations[search_perm-1][k]+'.csv')
                    temp_label = temp['label'].copy()

                    for lab_index in range(len(label)):
                        tagged_words = re.findall('<.*?:.*?>', temp_label[lab_index])
                        for tag in tagged_words:
                            striped = tag.strip('<>').split(':')
                            word = ":".join(striped[:-1])
                            label[lab_index] = label[lab_index].replace(word, tag)
            if k == 0:
                new_df = pd.DataFrame({'doc':doc, 'label':label})
            else:
                new_df = pd.concat([new_df, pd.DataFrame({'doc':doc, 'label':label})], ignore_index=True)
        new_df.to_csv('./full_cl_data/perm_'+str(perm)+'/step'+str(step+1)+'.csv', index=False)
                